# KoGPT2 - 가사 생성하기

## 샘플 코드 실습하기

In [1]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>', 
    eos_token='</s>', 
    unk_token='<unk>',
    pad_token='<pad>', 
    mask_token='<mask>'
)
output = tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")
print(output)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']


In [2]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


## 1. 데이터 불러오기

In [3]:
import pandas as pd 

data = pd.read_csv("music_labeling.csv", encoding="CP949", index_col=0)
print(data.shape)
data.head()

(1115, 5)


,title,singer,jenre,lyrics,emotion
0,Bad Girl Good Girl,미쓰에이,댄스,앞에선 한마디도 못하더니 뒤에선 내 얘길 안 좋게 해 참 어이가 없 나 같은 여잔 ...,분노
1,잔소리 (With 2AM 슬옹),아이유,발라드,늦게 다니지좀 마 술은 멀리좀 해봐 열살짜리 애처럼 말을 안듣니 정말 웃음만 나와 ...,기쁨
2,죽을 만큼 아파서 (Feat. 멜로우),MC몽,랩/힙합,너의 집 앞으로 가고 있어 빨리 전화 받어 내가 아파서 죽을 것 같거든 너 못보면 ...,슬픔
3,못해 (Feat. 美),포맨 (4MEN),발라드,이제는 밥을 먹어도 눈물 없이는 삼키지 못해 억지로 먹고 먹어도 속이 늘 허전해 노...,슬픔
4,죽어도 못 보내,2am,발라드,어려도 아픈건 똑같아 세상을 잘 모른다고 아픈걸 모르진 않아 괜찮아 질거라고 왜 거...,슬픔


### 데이터 저장 및 불러오기

In [4]:
# 미션
## 1115개의 "lyrics" 데이터를 하나로 합친다("\n\n\n"으로 구분)

"""
노래1 가사


노래2 가사


노래3 가사


...
"""
## kogpt2_lyrics_data.txt 로 저장하기

'\n노래1 가사\n\n\n노래2 가사\n\n\n노래3 가사\n\n\n...\n'

In [5]:
lyrics = "\n\n\n".join(data["lyrics"].tolist())

In [6]:
# 파일 쓰기
file_path = "./kogpt2_lyrics_data.txt"

with open(file_path, "w", encoding="utf-8") as f:
    f.write(lyrics)

In [7]:
# 파일 열기
file_path = "./kogpt2_lyrics_data.txt"

with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

print(lines[:5])

lyrics = " ".join([line.strip() for line in lines])

['앞에선 한마디도 못하더니 뒤에선 내 얘길 안 좋게 해 참 어이가 없 나 같은 여잔 처음 으로 으로 으로 본 것 같은데 왜 나를 판단하니 내가 혹시 두려운 거니 겉으론 속으론 나를 잘 알지도 못하면서 내 겉모습만 보면서 한심한 여자로 보는 너의 시선이 난 너무나 웃겨 춤출 땐 사랑은 춤추는 내 모습을 볼 때는 넋을 놓고 보고서는 끝나니 손가락질하는 그 위선이 난 너무나 웃겨 이런 옷 이런 머리 모양으로 이런 춤을 추는 여자는 뻔해 하 네가 더 뻔해 자신 없으면 저 뒤로 뒤로 뒤로 물러서면 되지 왜 자꾸 떠드니 네 속이 훤히 보이는 건 아니 겉으론 속으론 나를 잘 알지도 못하면서 내 겉모습만 보면서 한심한 여자로 보는 너의 시선이 난 너무나 웃겨 춤출 땐 사랑은 춤추는 내 모습을 볼 때는 넋을 놓고 보고서는 끝나니 손가락질하는 그 위선이 난 너무나 웃겨 날 감당 할 수 있는 남잘 찾아요 진짜 남자를 찾아요 말로만 남자다운 척할 남자 말고 날 불안해 하지 않을 남잔 없나요 자신감이 넘쳐서 내가 나일 수 있게 자유롭게 두고 멀리서 바라보는 겉으론 속으론 나를 잘 알지도 못하면서 내 겉모습만 보면서 한심한 여자로 보는 너의 시선이 난 너무나 웃겨 춤출 땐 사랑은 춤추는 내 모습을 볼 때는 넋을 놓고 보고서는 끝나니 손가락질하는 그 위선이 난 너무나 웃겨\n', '\n', '\n', '늦게 다니지좀 마 술은 멀리좀 해봐 열살짜리 애처럼 말을 안듣니 정말 웃음만 나와 누가 누굴보고 아이라 하는지 정말 웃음만 나와 싫은 얘기 하게 되는 내 맘을 몰라 좋은 얘기만 나누고 싶은 내맘을 몰라 그만할까 그만하자 하나부터 열까지 다 널 위한 소리 내 말 듣지 않는 너에게는 뻔한 잔소리 그만하자 그만하자 사랑하기만해도 시간 없는데 머리 아닌 가슴으로 하는 이야기 니가 싫다 해도 안 할수가 없는 이야기 그만하자 그만하자 나의 잔소리가 들려 밥은 제 때 먹는지 여잔 멀리 하는지 온 종일을 네 옆에 있고 싶은데 내가 그 맘인거야 주머니속에 널 넣고 다니면 정말 행복할텐데 둘이 아니면 안

In [8]:
# 결론: 가사를 한줄로 합친다.

## 2. 학습 데이터 만들기

In [9]:
import torch 
from torch.utils.data import Dataset, DataLoader 

# 목표: [i:i+block_size]를 쪼갠 텍스트를 tokenizer로 바꾼다.
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, block_size):
        """클래스를 변수에 저장하면 실행되는 곳"""
        self.data = data 
        self.tokenizer = tokenizer
        self.block_size = block_size

        # 방법 1. 텍스트를 쪼갠다 > 토크나이저에 넣는다
        # (선택) 방법 2. 토크나이저에 넣는다 > 텐서를 쪼갠다
        tokenized_data = self.tokenizer.encode(data)
        self.texts = []
        self.tokenized_texts = []
        for i in range(0, len(tokenized_data) - block_size, block_size):
            self.texts.append(data[i:i+block_size])
            self.tokenized_texts.append(tokenized_data[i:i+block_size])

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        return torch.tensor(self.tokenized_texts[idx], dtype=torch.long)

In [10]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>', 
    eos_token='</s>', 
    unk_token='<unk>',
    pad_token='<pad>', 
    mask_token='<mask>'
)

text = "늦게 다니지좀 마"
input_ids = tokenizer.encode(text, return_tensors='pt')
print(input_ids)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


tensor([[26187, 14841,  8263,  8217,  9109]])


In [11]:
train_data = TextDataset(lyrics, tokenizer, block_size=128)
# train_data.texts[:5]
print(len(train_data))
print(train_data[0].shape)
print(train_data[0])

2458
torch.Size([128])
tensor([12528,  7791, 29467,  7235,  9350,  8702, 18952, 10371,  7791,  9094,
        31187,  6962,  9183, 36846,  9098,  9338,  9105,  9760,  9126,  9063,
         9239,  9093,  8161,  9609,   739,  9021,   739,  9021,   739,  9021,
         9269,  9031,  9239,  7220, 10401, 24692, 12553, 19202, 17582, 10039,
         7888, 15087,  8094,  9122,  7172, 13095,  8133,  7428,  9238,  8133,
         7428, 24692,  9443,  9181, 10978,  9350,  9448,  9094, 13095, 36101,
         7489, 31266,  9036, 18273,  9093,  9828, 11867, 10099,  8143,  9039,
        10886,  9769, 25793, 14042,  6874, 12348,  8400,   739,  7314, 46260,
        12348, 15239,  9094, 11066,  9580, 10212, 32537,  8137, 12700, 10056,
         9449, 13585,  7172, 16549,  8267,  9033,  9022,  9056, 10886,  9769,
        25793, 14042,  6874, 10165, 11019, 10165, 11980, 13541, 10165, 19123,
        36769, 27497, 34200,  8711,  9078,  9873,  6824,  9267, 34200,  8711,
         9415, 20295,  9265, 15959, 15959

In [12]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
for x in train_loader:
    print(x)
    break

tensor([[13158,  7661, 15538,  ...,  7088,  9063, 12665],
        [ 9267,  9510, 17104,  ...,  9025,  9846,  6969],
        [13701, 40303, 31629,  ..., 10270,  8711, 19526],
        ...,
        [10724,  7265, 10489,  ...,  8159,  9054, 17003],
        [ 9393, 12821, 10125,  ..., 30328, 42656, 13358],
        [ 9512, 25558,  8162,  ...,  9848,  7481,  6969]])


## 3. 학습하기

In [13]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = model.to(device)

In [15]:
from torch.optim import AdamW 

epochs = 5
optimizer = AdamW(model.parameters(), lr=1e-4)

# 학습 시작
loss_history = []
print("Start!")
for epoch in range(epochs):
    #### Train #### 
    model.train()

    loss_train = 0.0
    for step, batch in enumerate(train_loader):
        batch = batch.to(device)

        optimizer.zero_grad()
        outputs = model(batch, labels=batch)
        loss = outputs.loss 
        loss.backward()
        optimizer.step() 

        loss_train += loss.item() * batch_size
        if step % 10 == 0:
            print(f"\tStep: {step} Loss: {loss.item()}") 
    
    loss_history.append(loss_train / len(train_data))

    print(f"Epoch: {epoch} Train Loss: {loss_train / len(train_data):.4f}")

Start!
	Step: 0 Loss: 4.922091484069824
	Step: 10 Loss: 4.651860237121582
	Step: 20 Loss: 4.495293617248535
	Step: 30 Loss: 4.4806623458862305
	Step: 40 Loss: 4.201980113983154
	Step: 50 Loss: 4.3089118003845215
	Step: 60 Loss: 4.287252902984619
	Step: 70 Loss: 4.096987247467041
Epoch: 0 Train Loss: 4.5224
	Step: 0 Loss: 3.9881746768951416
	Step: 10 Loss: 4.028519153594971
	Step: 20 Loss: 3.932128667831421
	Step: 30 Loss: 4.00371789932251
	Step: 40 Loss: 3.7731826305389404
	Step: 50 Loss: 3.8677496910095215
	Step: 60 Loss: 3.9714150428771973
	Step: 70 Loss: 3.8726799488067627
Epoch: 1 Train Loss: 3.8608
	Step: 0 Loss: 3.425788640975952
	Step: 10 Loss: 3.518324375152588
	Step: 20 Loss: 3.5558743476867676
	Step: 30 Loss: 3.555835247039795
	Step: 40 Loss: 3.66160249710083
	Step: 50 Loss: 3.4587297439575195
	Step: 60 Loss: 3.4925034046173096
	Step: 70 Loss: 3.4508895874023438
Epoch: 2 Train Loss: 3.4951
	Step: 0 Loss: 3.081448554992676
	Step: 10 Loss: 3.1452994346618652
	Step: 20 Loss: 3.2

In [16]:
# 토크나이저 저장할 경로 지정 (예: "./kogpt2-finetuned")
save_path = "./kogpt2-finetuned"
tokenizer.save_pretrained(save_path)

print(f"토크나이저가 {save_path} 경로에 저장되었습니다.")

토크나이저가 ./kogpt2-finetuned 경로에 저장되었습니다.


In [17]:
from transformers import PreTrainedTokenizerFast

# 저장한 경로에서 토크나이저 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained("./kogpt2-finetuned")

In [18]:
text = '네 마지막 말은' 
input_ids = tokenizer.encode(text, return_tensors='pt')

# 2. 새로운 토큰을 max_new_tokens 만큼 생성
max_new_tokens = 128  # 추가로 생성할 토큰 개수
generated = input_ids.to(device)  # 초기 입력 시퀀스

model = model.to(device)
with torch.no_grad():
    for _ in range(max_new_tokens):
        # 모델에 현재 시퀀스를 입력하여 출력(logits) 얻기
        outputs = model(generated)
        logits = outputs.logits  # shape: [batch_size, sequence_length, vocab_size]
        # print(logits)
        
        # 마지막 토큰의 logits만 추출 (다음 토큰 예측을 위해)
        next_token_logits = logits[:, -1, :]
        # print(next_token_logits)
        
        # Greedy 방식: 가장 확률이 높은 토큰 선택
        next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
        # print(next_token)
        
        # 생성된 토큰을 시퀀스에 추가
        generated = torch.cat((generated, next_token), dim=1)
        # print(generated.shape)

        print(tokenizer.decode(generated[0], skip_special_tokens=True))
        
        # (옵션) 만약 EOS 토큰(종료 토큰)이 생성되면 중단할 수 있음
        # if next_token.item() == tokenizer.eos_token_id:
        #     break
# print(generated.shape)
# print(tokenizer.decode(generated[0], skip_special_tokens=True))

네 마지막 말은 그
네 마지막 말은 그댈
네 마지막 말은 그댈 위해
네 마지막 말은 그댈 위해요
네 마지막 말은 그댈 위해요 그
네 마지막 말은 그댈 위해요 그댈
네 마지막 말은 그댈 위해요 그댈 위해
네 마지막 말은 그댈 위해요 그댈 위해요
네 마지막 말은 그댈 위해요 그댈 위해요 그
네 마지막 말은 그댈 위해요 그댈 위해요 그댈
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위해요
네 마지막 말은 그댈 위해요 그댈 위해요 그댈 위해요 그댈 위

In [19]:
print("pad_token:", tokenizer.pad_token, tokenizer.pad_token_id)
print("bos_token:", tokenizer.bos_token, tokenizer.bos_token_id)
print("eos_token:", tokenizer.eos_token, tokenizer.eos_token_id)
print("unk_token:", tokenizer.unk_token, tokenizer.unk_token_id)

pad_token: <pad> 3
bos_token: </s> 1
eos_token: </s> 1
unk_token: <unk> 5


In [21]:
import torch.nn.functional as F

text = '네 마지막 말은' 
input_ids = tokenizer.encode(text, return_tensors='pt')

# 2. 새로운 토큰을 max_new_tokens 만큼 생성
max_new_tokens = 128  # 추가로 생성할 토큰 개수
generated = input_ids.to(device)  # 초기 입력 시퀀스

model = model.to(device)

temperature = 0.7  # 다양성 조절 (낮으면 보수적, 높으면 창의적)
top_k = 50         # top-k 샘플링 (상위 k개 토큰만 고려)
with torch.no_grad():
    for _ in range(max_new_tokens):
        outputs = model(generated)
        logits = outputs.logits  # shape: [batch_size, sequence_length, vocab_size]
        
        # 마지막 토큰에 대한 logits 추출
        next_token_logits = logits[:, -1, :] / temperature
        
        # Top-k 필터링 적용
        top_k = min(top_k, next_token_logits.size(-1))
        indices_to_remove = next_token_logits < torch.topk(next_token_logits, top_k)[0][..., -1, None]
        next_token_logits[indices_to_remove] = -float("Inf")
        
        # 확률 분포 생성 후 샘플링
        probabilities = F.softmax(next_token_logits, dim=-1)
        next_token = torch.multinomial(probabilities, num_samples=1)
        
        # 생성된 토큰을 시퀀스에 추가
        generated = torch.cat((generated, next_token), dim=1)

        print(tokenizer.decode(generated[0], skip_special_tokens=True))
        
        # (옵션) EOS 토큰이 나오면 종료할 수 있습니다.
        # if next_token.item() == tokenizer.eos_token_id:
        #     break

# print(generated.shape)
# print(tokenizer.decode(generated[0], skip_special_tokens=True))

네 마지막 말은 내가
네 마지막 말은 내가 널
네 마지막 말은 내가 널 사랑
네 마지막 말은 내가 널 사랑할
네 마지막 말은 내가 널 사랑할 수
네 마지막 말은 내가 널 사랑할 수 있다는
네 마지막 말은 내가 널 사랑할 수 있다는 걸
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑할
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑할 수
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑할 수 있다는
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑할 수 있다는걸
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑할 수 있다는걸 잊지
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑할 수 있다는걸 잊지 말아
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑할 수 있다는걸 잊지 말아요
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑할 수 있다는걸 잊지 말아요 
네 마지막 말은 내가 널 사랑할 수 있다는 걸 네게 닿기를 마지막 말은 내가 널 사랑할 수 있다는걸 잊지 말아요  
네 마지막 말은 내가

In [20]:
text = '나는 내가 빛나는' 
input_ids = tokenizer.encode(text, return_tensors='pt')
print(input_ids)
print(tokenizer.pad_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.bos_token_id)
gen_ids = model.generate(input_ids.to(device),
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True
)
generated = tokenizer.decode(gen_ids[0])
print(generated)

tensor([[12102, 17582, 31408]])
3
1
1
나는 내가 빛나는 별이 되고 빛이 되어주기를 내 손을 잡아요 그댄 나의 영원한 꿈나무야 그대여 영원토록 널 사랑할게 넌 나만의 별이야 이 세상에 가장 아름다운 사람 아님 어디에도 없을 너를 원해 난 여기 있어줄래 날 봐줘 영원히 약속해줄꺼라 말해준다면 어떨까 우리 함께 걸어가기엔 너무 멀어 아쉬워도 기다려 줄께 나를 따라와 주어서 고마웠어 니가 내게로 와 주기를 바랐던 그때보다 훨씬 오래 잡은 손끝에 닿길 바라왔었기에 이제껏 못했던 말들이 다 잊혀질때까지 기다린다고 언젠
